<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/nlp-zurich/03-gradio-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Transformers demo with Gradio

## References

* https://huggingface.co/blog/gradio-spaces
* https://huggingface.co/blog/gradio

In [ ]:
# !pip install transformers gradio sentencepiece

## Example 1: Using the Transformers pipeline

In [1]:
import gradio as gr
from transformers import pipeline

In [2]:
pipe = pipeline("text-classification", model="lewtun/xlm-roberta-base-finetuned-marc-en")

Downloading:   0%|          | 0.00/976 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
pipe("The Lord of the Rings is waaay too long to read!!")

[{'label': 'poor', 'score': 0.43474119901657104}]

In [7]:
label2emoji = {"terrible": "💩", "poor": "😾", "ok": "🐱", "good": "😺", "great": "😻"}

def predict(text):
    preds = pipe(text)[0]
    return label2emoji[preds["label"]], round(preds["score"], 5)

predict("I love this soccer ball")

('😻', 0.82918)

In [10]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Predicting review scores from customer reviews",
    description="Enter some review text about an Amazon product and check what the model predicts for it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


KeyboardInterrupt: 

## Example 2: Using the Inference API from the Hugging Face Hub

In [ ]:
from huggingface_hub import InferenceApi

text = "My favourite book is Cryptonomicon!"
inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
preds = inference(inputs=text)
preds[0]

In [ ]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

In [ ]:
def inference_predict(text):
    inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [ ]:
inference_predict("I love these shoes")

In [ ]:
gradio_ui = gr.Interface.load(
    name="lewtun/xlm-roberta-base-finetuned-marc",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some text and check if model detects it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["I love these running shoes"], ["J'adore ces chaussures de course"], ["Ich liebe diese Laufschuhe"]
    ],
)

gradio_ui.launch(debug=True)